# Overview

The overall steps are the following:

1. Run a local https server for the application (dataUploader)
2. Click on the vafs link below to obtain a code.
3. Send this code to LumiNUS to exchange for an access token.
4. Use the access token within the LumiNUS API queries.

The above workflow ensures that the access token used corresponds to the permissions and privileges of the authenticated users. Anyone can use the dataUploader, but what they can do with it depends on their accorded permissions with LumiNUS.

# Setting up https server

In order to set up a local https server (as opposed to a http server), we need some self-signed certificates. Follow the instructions [here](https://codeburst.io/running-local-development-server-on-https-c3f80197ac4f) if you wish to generate your own certificates.

If you don't wish to generate your own, just use the ones in the repo.

In [200]:
from http.server import HTTPServer,BaseHTTPRequestHandler
import ssl, threading, ast
import ipywidgets as widgets
import http.client, urllib.request, urllib.parse, urllib.error, base64

In [179]:
class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):

    def do_GET(self):
        self.send_response(200)
        self.end_headers()
        #print("Hello")
        #print(self.headers.keys())
        print(self.requestline)
        self.wfile.write(b'Hello, world! Https server is working!')

In [180]:
httpd = HTTPServer(('localhost', 8080), SimpleHTTPRequestHandler)
httpd.allow_reuse_address = True

httpd.socket = ssl.wrap_socket(httpd.socket, keyfile = "cert_stuff/key.pem",
                               certfile="cert_stuff/cert.pem", server_side=True)

In [181]:
t1  = threading.Thread(target=httpd.serve_forever, daemon=True)

try:
    #httpd.serve_forever()
    t1.run()
except KeyboardInterrupt:
    httpd.shutdown()
finally:
    httpd.server_close()
    
# Use I I to send the keyboard interrupt signal

127.0.0.1 - - [13/Mar/2021 23:47:48] "GET /dataUploader?code=9RRlpckOCken5eIsfqavzQ.cjJUWjfm2Aj0K7hQJTZXCtrPwiA.U9t29PnbFoLJFYvXA2-qoiRFPlIG8jytZe3mARBCi4QCyR_o_gZx8SI4ku3kI_1nCRHy0rYCm9LWKSgDR4uGG7vFfRnBMHimdg-nCq8LI5CxUluyxUQMFfIAdKyU9eIt9-J2rNbI_2HZS-hUQ9hHLnkDZpU8zVfPeCUUkTtzmxBNMkoem7Auai5q-bYY-vBeDlG36pqgZZ6DlRWF4HO3t2t_hI7quvuBGzCGZOpw4Wd1G0FYEDAyhQnF34nH_qGpWpCY8Aanim4zlweGut0W3psAqOrBzfj_budxUkSHxtZDZUZ7SjDWzNUIe8CSrixc8itfVjr4L-gpac0w0edkBg HTTP/1.1" 200 -


GET /dataUploader?code=9RRlpckOCken5eIsfqavzQ.cjJUWjfm2Aj0K7hQJTZXCtrPwiA.U9t29PnbFoLJFYvXA2-qoiRFPlIG8jytZe3mARBCi4QCyR_o_gZx8SI4ku3kI_1nCRHy0rYCm9LWKSgDR4uGG7vFfRnBMHimdg-nCq8LI5CxUluyxUQMFfIAdKyU9eIt9-J2rNbI_2HZS-hUQ9hHLnkDZpU8zVfPeCUUkTtzmxBNMkoem7Auai5q-bYY-vBeDlG36pqgZZ6DlRWF4HO3t2t_hI7quvuBGzCGZOpw4Wd1G0FYEDAyhQnF34nH_qGpWpCY8Aanim4zlweGut0W3psAqOrBzfj_budxUkSHxtZDZUZ7SjDWzNUIe8CSrixc8itfVjr4L-gpac0w0edkBg HTTP/1.1


While the server is running, click on this link to authenticate with LumiNUS:

[Authentication link](https://vafs.nus.edu.sg/adfs/oauth2/authorize?response_type=code&client_id=INC000002292056&redirect_uri=https://localhost:8080/dataUploader&resource=sg_edu_nus_oauth)

The code will be returned to the server above. Copy the code and save it using the widget below. You can kill the http server now.

# Extracting code

In [197]:
code = widgets.Text(value='', placeholder='Paste code here', description='code:')
code

Text(value='', description='code:', placeholder='Paste code here')

# Exchanging for Token

The next step is to use the code value to exchange for a token which can then be used with the API. The corresponding help page on the developer API is this [one](https://luminus.portal.azure-api.net/docs/services/Login/operations/GetADFSAccessToken?).

In [184]:
headers = {
    'Ocp-Apim-Subscription-Key': 'e02ac553a8ed4da7a3762fc8195b4db0',
    'Content-Type': 'application/x-www-form-urlencoded'
}

In [185]:
body = 'grant_type=authorization_code&client_id=INC000002292056&resource=sg_edu_nus_oauth&code=' + \
       code.value + '&redirect_uri=https://localhost:8080/dataUploader'
# body

In [187]:
conn = http.client.HTTPSConnection('luminus.azure-api.net')
conn.request("POST", "/login/ADFSToken", body, headers)
response = conn.getresponse()
data = response.read()
#print(data)

In [203]:
data_dict = ast.literal_eval(data.decode("UTF-8"))
a_token = data_dict["access_token"]

# Calling LumiNUS API


## Getting User Profile

The reference page for this request is [this one](https://luminus.portal.azure-api.net/docs/services/User/operations/GetMyProfile/console).

In [216]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': 'e02ac553a8ed4da7a3762fc8195b4db0',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {a_token}'
}

In [217]:
conn = http.client.HTTPSConnection('luminus.azure-api.net')
# conn.request("GET", "/user/Profile?%s" % params, "{body}", headers)
conn.request("GET", "/user/Profile", headers=headers)
response = conn.getresponse()
data = response.read()

In [227]:
data_dict = ast.literal_eval(data.decode('UTF-8').replace('true', 'True'))

In [228]:
data_dict

{'id': '40eb7a34-798c-4f7e-bb18-83f69d8c7266',
 'userID': 'stavg',
 'userName': 'Vik',
 'userNameOriginal': 'Vikneswaran S/O Gopal',
 'userMatricNo': '047541',
 'userFaculty': 'STATISTICS & APPLIED PROBABILITY',
 'nickName': 'Vik',
 'mobileNo': '',
 'officialEmail': 'stavg@nus.edu.sg',
 'email': 'stavg@nus.edu.sg',
 'displayPhoto': True,
 'aboutMe': '-'}

In [229]:
conn.close()

## Files API call

I would like to be able to use [this call](https://luminus.portal.azure-api.net/docs/services/Files/operations/AddFolder?) to set access rights to individual files.